# Autocombat TOP+StrokeMRI, SABREand Insight46 harmonized datasets

This notebook creates datasets from the above hamornized as if each of 1. a blended TOP and StrokeMRI 2.SABRE and 3. Insight  is indpendant and from a different "batch" (TOP and StrokeMRI are same acquisition on same machine, so same batch)

Note this must be run in the `comscan` ( `comscan6`) environment

## import libraries

In [ ]:
import os

import pandas as pd
import numpy as np



In [ ]:
import ComScan


## Let's see how we would apply this to our data
We will have to flip it on it's side to make it work

In [ ]:
!python --version

In [ ]:
filepath = '../open_work/internal_results/cleaned_pvc2s/' 
filename_mri = os.path.join(filepath,'StrokeMRI_pvc2c.csv') 
filename_top = os.path.join(filepath,'TOP_pvc2c.csv') 
filename_sabre = os.path.join(filepath,'SABRE_pvc2_cleaned.csv') 
filename_insight46 =  os.path.join(filepath,'Insight46_pvc2c.csv') 

In [ ]:
# Now we must clean SABRE down to be like the other datasets, including dropping NAns
MRI = pd.read_csv(filename_mri)
TOP = pd.read_csv(filename_top)
Insight46 = pd.read_csv(filename_insight46)

SABRE =  pd.read_csv(filename_sabre)
coly = TOP.columns
SABRE = SABRE[coly]
SABRE= SABRE.dropna()
SABRE.tail(3)

In [ ]:
TOP['site'] = 0
MRI['site'] = 1
MRI

In [ ]:
#TOP.age.describe()

In [ ]:
# Here we will harmonize just TOP and StrokeMRI

In [ ]:
TOPMRI = pd.concat([TOP, MRI])
TOPMRI 

In [ ]:
# data = pd.DataFrame([{"features_1": 0.97, "features_2": 2, "sites": 0},
# {"features_1": 1.35, "features_2": 1.01, "sites": 1}, {"features_1": 1.43, "features_2": 1.09, "sites": 1},
# {"features_1": 0.85, "features_2": 2.3, "sites": 0}])
# data
sex_mapping = {'F':0,'M':1}
TOPMRI = TOPMRI.assign(sex = TOPMRI.sex.map(sex_mapping))

TOPMRI 

In [ ]:
# TOPMRI= TOPMRI.reset_index()
# TOPMRI[]

## Now very importantly, we have our sites pre-defined. This means that the AutoCOmbat module will execute it's own version of combat (without clustering). Therefore we can implement the Autocombat neurocombat module's combat, without a need for clustering
If you don't believe me just read the documentation and the code itself

classComScan.neurocombat.Combat(features, sites, discrete_covariates=None, continuous_covariates=None, ref_site=None, empirical_bayes=True, parametric=True, mean_only=False, return_only_features=False, raise_ref_site=True, copy=True)

In [ ]:
TOPMRI['decade']=(TOPMRI['age']/10).round()
TOPMRI['decade']

In [ ]:
# TOPMRI['decade']=(TOPMRI['age']/10).round()
# TOPMRI['decade']

In [ ]:
TOPMRI['decade'].unique()

In [ ]:
#auto_combat = ComScan.neurocombat.AutoCombat(features=["features_1"], sites_features=["site_features_0", "site_features_1"],
#continuous_cluster_features=["site_features_0", "site_features_1"], size_min=2)
combat = ComScan.neurocombat.Combat(features=['gm_vol', 'wm_vol',
       'csf_vol', 'gm_ivc_ratio', 'gmwm_ivc_ratio', 'wmh_vol', 'wmh_count',
       'deepwm_b_cov', 'aca_b_cov', 'mca_b_cov', 'pca_b_cov', 'totalgm_b_cov',
       'deepwm_b', 'aca_b', 'mca_b', 'pca_b', 'totalgm_b',], sites=[ "site"], discrete_covariates=['sex'],continuous_covariates=['decade'],)


In [ ]:
print(combat.fit(TOPMRI))

In [ ]:
print(combat.gamma_star_)

In [ ]:
print(combat.transform(TOPMRI))

In [ ]:
TOPMRI

In [ ]:
transformed_TOPMRI= combat.transform(TOPMRI)
transformed_TOPMRI

In [ ]:
TOP_transformed = transformed_TOPMRI[ transformed_TOPMRI['site']==0]
TOP_transformed = TOP_transformed.drop(['site', 'decade', 'Unnamed: 0'], axis=1)
TOP_transformed

In [ ]:
MRI_transformed = transformed_TOPMRI[ transformed_TOPMRI['site']==1]
MRI_transformed = MRI_transformed.drop(['site', 'decade', 'Unnamed: 0'], axis=1)
MRI_transformed

In [ ]:
TOP_transformed.to_csv('../open_work/internal_results/autocombat/autocom_harm_top1.csv')

In [ ]:
MRI_transformed.to_csv('../open_work/internal_results/autocombat/autocom_harm_mri1.csv')